In [5]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import glob
import pylab as pl
from sklearn.cluster import KMeans
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import tree 
from sklearn import preprocessing
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix,roc_auc_score,roc_curve
import matplotlib.pyplot as plt

In [6]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Part 1: Text cleaning

In [7]:
News_DF=pd.read_csv('News_Headline.csv', error_bad_lines=False, encoding='latin-1')

In [8]:
News_DF = News_DF.drop(columns=['Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11'])

## split the datasets

In [24]:
from sklearn.model_selection import train_test_split
News_DF['cata'] = pd.factorize(News_DF['category'])[0] 
Xtrain,Xtest ,ytrain,ytest = train_test_split(News_DF['headline'], News_DF['cata'], random_state = 2020, test_size = 0.2)

# Part 7: BERT

In [14]:
from transformers import DistilBertTokenizerFast
from tqdm import tqdm
import torch
from transformers import DistilBertForSequenceClassification, \
  AdamW, get_linear_schedule_with_warmup
from datasets import load_metric

batch_size = 15
epochs = 2
lr_init = 1e-5
max_len = 256
warmup_steps = 3


K = 9



##Create batched inputs using Huggingface's DistilBERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
def batch_data(data, labels, bsize):
    batches = []
    sentences = data.tolist()
    labels = labels.tolist()
    for i in range(0, len(sentences), bsize):
        s = sentences[i: i + bsize]
        Y = labels[i: i + bsize]
        X = tokenizer.batch_encode_plus(
            s, max_length=max_len, padding='longest', truncation=True,
            return_attention_mask=True, return_token_type_ids=False)
        batches.append((X, Y, s))
    return batches


train_batches = batch_data(Xtrain, ytrain, bsize=batch_size)
test_batches = batch_data(Xtest, ytest, bsize=batch_size)

model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-cased', 
    num_labels=K, 
    output_hidden_states=True)

if torch.cuda.is_available():
    device = torch.device('cuda:0') # GPU
else:
    device = torch.device('cpu') # CPU
model.to(device)


optimizer = AdamW(model.parameters(), lr=lr_init)
lr = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, 
    num_training_steps=len(train_batches))


def runner(batches, desc: str, train=True):
    
    grad_mode = torch.enable_grad if train else torch.no_grad
    preds = []
    
    if train:
        model.train()
    else:
        model.eval()
    
    for epoch in range(epochs if train else 1):
        
        acc = load_metric("accuracy",keep_in_memory=True)
        f1 = load_metric("f1",keep_in_memory=True)
        precision= load_metric("precision",keep_in_memory=True)
        recall= load_metric("recall",keep_in_memory=True)

        cumloss = 0.0
        embeds = []
        
        with tqdm(total=len(batches)) as bar:

            for i, batch in enumerate(batches):
                X, Y, _ = batch
                inputs = torch.tensor(X['input_ids'], device=device)
                attmsk = torch.tensor(X['attention_mask'], device=device)
                labels = torch.tensor(Y, device=device)
                batch = {'input_ids': inputs,
                         'attention_mask': attmsk,
                         'labels': labels}
                with grad_mode():
                    outputs = model(**batch)
                    embeds.append(outputs[-1][1][:, 0, :].squeeze().detach().cpu())
                    loss = outputs.loss
                    if train:
                        loss.backward()
                        optimizer.step()
                        lr.step()
                        optimizer.zero_grad()
                    logits = outputs.logits
                    Yhat = torch.argmax(logits, dim=-1)
                    preds.append(Yhat)
                    cumloss += loss.clone().detach().cpu().item()
                    acc.add_batch(predictions=Yhat, references=Y)
                    f1.add_batch(predictions=Yhat, references=Y)
                    precision.add_batch(predictions=Yhat, references=Y)
                    recall.add_batch(predictions=Yhat, references=Y)

                bar.update(1)
            bar.set_description('epoch: %s, %s loss: %.5f, f1-score: %.5f, accuracy: %.5f, precision: %.5f, recall: %.5f' %
                                (epoch + 1, desc,
                                 cumloss / (i + 1),                
                                 f1.compute(average="macro")['f1'],
                                 acc.compute()['accuracy'],
                               precision.compute(average="macro")['precision'],
                               recall.compute(average="macro")['recall']))
                
    embeds = torch.cat(embeds, dim=0)
    return preds, embeds

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.w

In [15]:
runner(train_batches, 'train')

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

epoch: 1, train loss: 0.88823, f1-score: 0.72398, accuracy: 0.72437, precision: 0.72450, recall: 0.72432: 100%|██████████| 2396/2396 [05:23<00:00,  7.40it/s]
epoch: 2, train loss: 0.62453, f1-score: 0.80974, accuracy: 0.80984, precision: 0.81017, recall: 0.80979: 100%|██████████| 2396/2396 [05:22<00:00,  7.43it/s]


([tensor([8, 5, 8, 8, 3, 5, 8, 5, 3, 8, 8, 8, 8, 8, 8], device='cuda:0'),
  tensor([8, 3, 5, 8, 8, 8, 8, 8, 3, 8, 8, 3, 8, 3, 8], device='cuda:0'),
  tensor([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5, 8, 5, 8], device='cuda:0'),
  tensor([8, 8, 8, 8, 3, 8, 8, 8, 8, 8, 8, 5, 8, 8, 8], device='cuda:0'),
  tensor([8, 8, 8, 8, 8, 3, 8, 3, 8, 8, 8, 8, 8, 8, 5], device='cuda:0'),
  tensor([8, 8, 8, 8, 8, 3, 8, 8, 8, 8, 3, 5, 3, 8, 8], device='cuda:0'),
  tensor([8, 5, 5, 8, 5, 8, 8, 8, 8, 8, 3, 8, 8, 8, 8], device='cuda:0'),
  tensor([1, 5, 3, 8, 8, 8, 8, 1, 8, 5, 7, 1, 5, 8, 8], device='cuda:0'),
  tensor([8, 5, 8, 8, 8, 3, 8, 8, 1, 5, 5, 8, 8, 5, 8], device='cuda:0'),
  tensor([8, 8, 8, 8, 8, 7, 1, 8, 3, 8, 8, 8, 7, 7, 8], device='cuda:0'),
  tensor([5, 3, 5, 1, 1, 8, 8, 3, 8, 8, 8, 8, 8, 8, 8], device='cuda:0'),
  tensor([8, 8, 8, 5, 8, 8, 8, 8, 8, 8, 3, 7, 5, 1, 1], device='cuda:0'),
  tensor([1, 8, 8, 8, 5, 8, 8, 8, 8, 8, 5, 8, 8, 5, 3], device='cuda:0'),
  tensor([8, 5, 8, 1, 5, 8, 7, 1, 8, 8

vis

In [19]:
def save_matrix(matrix, fpath):
    D1, D2 = matrix.shape
    tsv = ""
    for i in range(D1):
        for j in range(D2):
            tsv += str(matrix.iloc[i, j]) + '\t'
        tsv = tsv.strip('\t') + '\n'
    tsv = tsv.strip('\n')
    with open(fpath, "w") as fd:
        fd.write(tsv)

preds_list, embeddings_vectors= runner(test_batches, 'test', train=False)
embeddings_vect= embeddings_vectors.tolist()
embs_df= pd.DataFrame(embeddings_vect)
save_matrix(embs_df, "test_embeddings.tsv")
save_matrix(Xtest, "Xtest.tsv")


epoch: 1, test loss: 0.68415, f1-score: 0.78636, accuracy: 0.78646, precision: 0.78738, recall: 0.78632: 100%|██████████| 599/599 [00:25<00:00, 23.73it/s]


https://projector.tensorflow.org/ 

## Part 8: LSTM

In [48]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers import Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
from tensorflow.keras import utils
n_most_common_words = 8000
tokenizer = Tokenizer(num_words=n_most_common_words, filters=string.punctuation, lower=True)
def convert(txt):
  txt_=[str (item) for item in txt.tolist()]
  tokenizer.fit_on_texts(txt_)
  sequences = tokenizer.texts_to_sequences(txt_)
  X = pad_sequences(sequences)
  return X

Xdata=convert(News_DF['headline'])
Xtrain,Xtest ,ytrain,ytest = train_test_split(Xdata, News_DF['cata'], random_state = 2020, test_size = 0.2)
ytrain= utils.to_categorical(ytrain, 9)
ytest= utils.to_categorical(ytest, 9)


In [49]:
# no of Epochs for Model Training
epochs = 12
# Output Dimensions for Embedded Layer
emb_dim = 256
# Batch size for training
batch_size = 256
# Creating a LSTM model
model = tf.keras.Sequential()
model.add(layers.Embedding(n_most_common_words, emb_dim, input_length=Xtrain.shape[1]))
model.add(layers.LSTM(512, activation='sigmoid'))
model.add(layers.Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(layers.Dense(9,activation='sigmoid'))
# compiling the layer 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())
# Training the model 
lstm = model.fit(Xtrain, ytrain, epochs=epochs, batch_size=batch_size, validation_split=0.1,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 66, 256)           2048000   
                                                                 
 lstm_2 (LSTM)               (None, 512)               1574912   
                                                                 
 dense_4 (Dense)             (None, 128)               65664     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 9)                 1161      
                                                                 
Total params: 3,689,737
Trainable params: 3,689,737
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/12
127/127 [===================

In [50]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
ypred= model.predict(Xtest)
#score = model.evaluate(Xtest,ytest, batch_size=batch_size, verbose=0)
ypred= np.argmax(ypred, axis=1)
ytest=np.argmax(ytest, axis=1)
accuracy = accuracy_score(ytest, ypred)
precision = precision_score(ytest, ypred,average="micro")
recall = recall_score(ytest, ypred,average="micro")
f1 = f1_score(ytest, ypred,average="micro")
print("acc:{}, pre:{}, recall:{}, f1:{}".format(accuracy,precision,recall,f1))

acc:0.7189935426408373, pre:0.7189935426408373, recall:0.7189935426408373, f1:0.7189935426408373
